In [1]:
#Memprediksi pasien yang rawan terkena penyakit stroke dengan menggunakan datasets yang sudah ada sebelumnya yang diambil dari kaggle

In [2]:
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt

In [3]:
# Membuka datasest csv kemudian menghapus kolom id dan juga kolom pernikahan karena rata-rata sudah menikah
df = pd.read_csv('Documents/Portofolio/datamentah/stroke.csv', delimiter = ',', header = 0)
df = df.drop(columns = ['id', 'ever_married'])
df

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,1,Private,Urban,228.69,36.6,formerly smoked,1
1,Female,61.0,0,0,Self-employed,Rural,202.21,NaN,never smoked,1
2,Male,80.0,0,1,Private,Rural,105.92,32.5,never smoked,1
3,Female,49.0,0,0,Private,Urban,171.23,34.4,smokes,1
4,Female,79.0,1,0,Self-employed,Rural,174.12,24.0,never smoked,1
...,...,...,...,...,...,...,...,...,...,...
5105,Female,80.0,1,0,Private,Urban,83.75,NaN,never smoked,0
5106,Female,81.0,0,0,Self-employed,Urban,125.20,40.0,never smoked,0
5107,Female,35.0,0,0,Self-employed,Rural,82.99,30.6,never smoked,0
5108,Male,51.0,0,0,Private,Rural,166.29,25.6,formerly smoked,0


In [4]:
#melihat kekosongan data
df.isna().sum()

gender                 0
age                    0
hypertension           0
heart_disease          0
work_type              0
Residence_type         0
avg_glucose_level      0
bmi                  201
smoking_status         0
stroke                 0
dtype: int64

In [5]:
#membersihkan kekosongan data dengan menghapus nilai Nan
df.dropna(inplace = True)
df.isna().sum()

gender               0
age                  0
hypertension         0
heart_disease        0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
dtype: int64

In [6]:
#mengubah data stirng menjadi bilangan integer agar pemodelan bisa dilakukan dengan mudah
df['gender'] = pd.factorize(df.gender)[0]
df['Residence_type'] = pd.factorize(df.Residence_type)[0]
df['smoking_status'] = pd.factorize(df.smoking_status)[0]
df['work_type'] = pd.factorize(df.work_type)[0]
df

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,0,67.0,0,1,0,0,228.69,36.6,0,1
2,0,80.0,0,1,0,1,105.92,32.5,1,1
3,1,49.0,0,0,0,0,171.23,34.4,2,1
4,1,79.0,1,0,1,1,174.12,24.0,1,1
5,0,81.0,0,0,0,0,186.21,29.0,0,1
...,...,...,...,...,...,...,...,...,...,...
5104,1,13.0,0,0,3,1,103.08,18.6,3,0
5106,1,81.0,0,0,1,0,125.20,40.0,1,0
5107,1,35.0,0,0,1,1,82.99,30.6,1,0
5108,0,51.0,0,0,0,1,166.29,25.6,0,0


In [7]:
#melihat persebaran data 
df.describe()

,gender,age,hypertension,heart_disease,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
count,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000,4909.000000
mean,0.590548,42.865374,0.091872,0.049501,0.842534,0.492768,105.305150,28.893237,1.583826,0.042575
std,0.492197,22.555115,0.288875,0.216934,1.124057,0.499999,44.424341,7.854067,1.090146,0.201917
min,0.000000,0.080000,0.000000,0.000000,0.000000,0.000000,55.120000,10.300000,0.000000,0.000000
25%,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,77.070000,23.500000,1.000000,0.000000
50%,1.000000,44.000000,0.000000,0.000000,0.000000,0.000000,91.680000,28.100000,1.000000,0.000000
75%,1.000000,60.000000,0.000000,0.000000,2.000000,1.000000,113.570000,33.100000,3.000000,0.000000
max,2.000000,82.000000,1.000000,1.000000,4.000000,1.000000,271.740000,97.600000,3.000000,1.000000


In [8]:
#memisahkan variabel bebas dan juga terikat dan membagi data tersebut untuk training model dan juga test mode;
X = df.drop('stroke', axis = 1)
y = df['stroke']
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size = 0.2,random_state = 1)

In [9]:
#menggunakan pemodelan Gaussian Naive Bayes untuk mengolah data
model = GaussianNB()
model.fit(X_train, y_train)

GaussianNB()

In [10]:
#memprediksi hasil dari pemodelan
y_pred = model.predict(X_test)

In [11]:
y_pred

array([0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,

In [12]:
#melihat apakah sama antara data luaran aslinya dengan data luaran dari pemodelan
y_test = y_pred

In [13]:
#melihat persentase keakuratan luaran dari data yang sudah di prediksi
r2_score(y_test, y_pred)

1.0

In [14]:
#memasukkan data pasien yang baru
newdata = np.array([[input('gender:'),
                     input('age:'),
                     input('hypertension:'),
                     input('heart disease:'),
                     input('work type:'),
                     input('Residence type'),
                     input('avg_glucose_level'),
                     input('bmi:'),
                     input('smoking_status')]],dtype=float)

gender:0
age:24
hypertension:1
heart disease:1
work type:0
Residence type1
avg_glucose_level221.2
bmi:25
smoking_status1


In [15]:
data2 = pd.DataFrame(newdata, columns=['gender', 'age', 'hypertension', 'heart_disease', 'work_type','Residence_type','avg_glucose_level', 'bmi', 'smoking_status'])

In [16]:
#memprediksi data dari pasien baru
#jika 0 maka memiliki potensi stroke rendah, sedangkan 1 memiliki potensi stroke yang tinggi
pred = model.predict(data2)
pred

array([1], dtype=int64)